#### **PyTorch**

In [1]:
import torch
import torch.nn.functional as F
print(f"PyTorch Version: {torch.__version__}")

import torch
print(f"Cude is available: {torch.cuda.is_available()}")
print(f"Device name: {torch.cuda.get_device_name(0)}")

PyTorch Version: 2.3.0+cu121
Cude is available: True
Device name: NVIDIA H100 PCIe


#### **Import Other Libraries**

In [64]:
from datasets import load_dataset 
from torch.utils.data import DataLoader
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
import numpy as np
import evaluate
import transformers
from transformers import TrainingArguments
import torch 
import matplotlib.pyplot as plt 
from transformers import DataCollatorWithPadding
import os 
from pathlib import Path
import random 
from datasets import Dataset, DatasetDict
import warnings
from functools import partial
from datasets import concatenate_datasets
from functools import partial 
from tqdm import tqdm 
import textwrap
from IPython.display import display
from IPython.display import Markdown
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM
from peft import LoraConfig, get_peft_model 
from transformers import BitsAndBytesConfig
import os 
import re 
os.environ["WANDB_DISABLED"] = "true"
warnings.filterwarnings('ignore', message='Was asked to gather along dimension 0, but all input tensors were scalars; will instead unsqueeze and return a vector.')
from llmft.train import evaluate_model, train_one_epoch, EarlyStopping
from llmft.metrics import compute_recall
from llmft.losses import FocalLoss
from llmft.utils import predict
from llmft.generate import generate_dataset
from llmft.paper import generate_recall_table

In [63]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")

Device: cuda


#### **Parameters**

In [3]:
# This cell is tagged with `parameters`
"meta-llama/Meta-Llama-3-8B-Instruct" #"google/gemma-1.1-7b-it" #microsoft/phi-2" #"microsoft/phi-2" #"#"meta-llama/Llama-2-7b-chat-hf" # "distilbert-base-uncased" 
model_name = "microsoft/Phi-3-mini-4k-instruct"
column = 'text'
epochs = 1
seed = 0
verbose = True 
test_size = 0.5
p = 0.0

#### **Qlora**

In [4]:
from peft import LoraConfig, get_peft_model 
from transformers import BitsAndBytesConfig

# ----- QUANTIZATION -------# 
# Activate 4-bit precision base model loading
use_4bit = True

# Compute dtype for 4-bit base models
bnb_4bit_compute_dtype = "float16"

# Quantization type (fp4 or nf4)
bnb_4bit_quant_type = "nf4"

# Activate nested quantization for 4-bit base models (double quantization)
use_nested_quant = True

# Load tokenizer and model with QLoRA configuration
compute_dtype = getattr(torch, bnb_4bit_compute_dtype)

# Enable fp16/bf16 training (set bf16 to True with an A100)
fp16 = False
bf16 = True

bnb_config = BitsAndBytesConfig(
    load_in_4bit=use_4bit,
    bnb_4bit_quant_type=bnb_4bit_quant_type,
    bnb_4bit_compute_dtype=compute_dtype,
    bnb_4bit_use_double_quant=use_nested_quant,
)

# Check GPU compatibility with bfloat16
if compute_dtype == torch.float16 and use_4bit:
    major, _ = torch.cuda.get_device_capability()
    if major >= 8:
        print("=" * 80)
        print("Your GPU supports bfloat16: accelerate training with bf16=True")
        print("=" * 80)

# ----- LORA -------# 

lora_config = LoraConfig(
    r=8,
    target_modules=["q_proj", "o_proj", "k_proj", "v_proj", "gate_proj", "up_proj", "down_proj"],
    task_type="CAUSAL_LM",
)

Your GPU supports bfloat16: accelerate training with bf16=True


#### **Instantiate Model**

In [9]:
model = AutoModelForCausalLM.from_pretrained(model_name, 
                                             device_map="auto", 
                                             quantization_config=bnb_config, 
                                             trust_remote_code=True)# So we can do gradient checkpointing
model.config.use_cache = False
model.config.pretraining_tp = 1
model.config.gradient_checkpointing = True
model.enable_input_require_grads()
print(model.generation_config)

model = get_peft_model(model, lora_config)
print(model.print_trainable_parameters())

`flash-attention` package not found, consider installing for better performance: No module named 'flash_attn'.
Current `flash-attenton` does not support `window_size`. Either upgrade or use `attn_implementation='eager'`.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": [
    32000,
    32001,
    32007
  ],
  "pad_token_id": 32000
}

trainable params: 4,456,448 || all params: 3,825,536,000 || trainable%: 0.11649212031987152
None


In [54]:
tokenizer = AutoTokenizer.from_pretrained(model_name)
tokenizer.pad_token = tokenizer.eos_token
def tokenizer_function(example):
  return {"formatted_chat": tokenizer.apply_chat_template(example["messages"], tokenize=True, add_generation_prompt=False)}



Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [59]:
lr = 1e-4
warmup_ratio = 0.
batch_size = 64
epochs = 30
patience = float('inf') 
gamma = 0.0

training_status = 'standard' if lr==1e-4 else 'preferred'

#### **Data set**

In [74]:
dataset = load_dataset("ppower1/chat_instrument")['train']
labels = np.array([1 if i[2]['content'] == 'Yes' else 0 for i in dataset['messages']])
dataset = dataset.select(range(1000))

# Reshuffle and split the combined dataset with a fixed seed
dataset = dataset.train_test_split(test_size=test_size, seed=seed)  # adjust test_size as needed
tokenized_dataset = dataset.map(tokenizer_function, batched=True)
tokenized_dataset = tokenized_dataset.remove_columns(['messages'])



DatasetDict({
    train: Dataset({
        features: ['formatted_chat'],
        num_rows: 500
    })
    test: Dataset({
        features: ['formatted_chat'],
        num_rows: 500
    })
})

In [70]:
train_loader = DataLoader(tokenized_dataset['train'], batch_size=batch_size, collate_fn=DataCollatorWithPadding(tokenizer), shuffle=True)
test_loader = DataLoader(tokenized_dataset['test'], batch_size=batch_size, collate_fn=DataCollatorWithPadding(tokenizer))

In [71]:
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
scheduler =transformers.optimization.get_linear_schedule_with_warmup(optimizer,int(warmup_ratio*len(train_loader)*epochs), len(train_loader)*epochs)

In [72]:

class_weights = torch.tensor([1., 1.], device=device) #torch.tensor([1-n_positive/n_samples, n_positive/n_samples], device=device)
criterion = FocalLoss(alpha=class_weights, gamma=gamma, mode='output')
print(criterion.gamma)

0.0


In [73]:
evaluation_losses = [evaluate_model(model, test_loader, criterion, device)]
training_losses = [evaluate_model(model, train_loader, criterion, device)]
early_stopping = EarlyStopping(patience=patience, verbose=True, path=model_name + 'checkpoint.pt')
metric_history = []
learning_rate_history = []

for epoch in range(epochs):

    train_loss, (recall, _), current_lr = train_one_epoch(model, train_loader, optimizer, scheduler, compute_recall, criterion, device)

    training_losses.append(train_loss)
    metric_history.append(recall)
    learning_rate_history.append(current_lr)
    
    val_loss = evaluate_model(model, test_loader, criterion, device)
    evaluation_losses.append(val_loss)
    
    # Call early stopping
    early_stopping(val_loss, model, epoch)
    if early_stopping.early_stop:
        print("***Early stopping***")
        print(f"Min Validation Loss: {early_stopping.best_epoch}")
        print(f"Current Epoch: {epoch}")
        break

model.load_state_dict(torch.load(model_name + 'checkpoint.pt'))

ValueError: You should supply an encoding or a list of encodings to this method that includes input_ids, but you provided ['formatted_chat']

In [15]:
trainer.train()

/home/ubuntu/llmft/llms/lib/python3.10/site-packages/torch/utils/checkpoint.py:464: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. In version 2.4 we will raise an exception if use_reentrant is not passed. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  warnings.warn(


Step,Training Loss,Validation Loss
1,2.332500,2.314873
2,2.335800,2.258834
3,2.299800,2.187922
4,2.199900,2.103839
5,2.089500,2.016492
6,2.055400,1.931477
7,1.922400,1.849380
8,1.834500,1.778381
9,1.798100,1.724100
10,1.725000,1.694108


In [ ]:
steps, train_loss =   [i['step'] for i in trainer.state.log_history if 'loss' in i],  [i['loss'] for i in trainer.state.log_history if 'loss' in i]
eval_loss = [i['eval_loss'] for i in trainer.state.log_history if 'eval_loss' in i]

In [ ]:
plt.plot(train_loss, label='Train')
plt.plot(eval_loss, label='Validation')
plt.legend()
plt.show()

In [ ]:
labels = np.array([1 if i[2]['content'] == 'Yes' else 0 for i in dataset['messages']])
sum(labels)

In [ ]:
selected = [dataset['messages'][i]  for i in range(len(dataset)) if labels[i] == 1]

In [ ]:
len(selected)

In [ ]:
for counter, message in enumerate(selected):
    tokenized_chat = tokenizer.apply_chat_template(message[:2], tokenize=True, add_generation_prompt=True, return_tensors='pt')
    outputs = trainer.model.generate(tokenized_chat, max_new_tokens=1)
    text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    print(counter, text.split("Prediction:")[1])